In [122]:
from langchain_core.tools import tool
from utils.resume_parser import ResumeParser
import PyPDF2
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langgraph.prebuilt import create_react_agent

In [123]:
load_dotenv()

True

In [124]:
resume_parser = ResumeParser()

In [125]:
llm = ChatGroq(model="openai/gpt-oss-20b")

In [126]:
resume_analysis_prompt = f"""
            From given Skills, Education and Experience, Analyze this resume information and provide specific, actionable suggestions 
            for improvement to make it more competitive in the job market.
            
            === RESUME DATA ===
            
            Skills: {skills}
            
            Education: {education}
            
            Experience: {experience}
            
            === ANALYSIS INSTRUCTIONS ===
            
            Provide a comprehensive analysis with the following clearly labeled sections:
            
            1. OVERALL ASSESSMENT
            • Strengths: Identify 3-5 strong aspects of the resume
            • Weaknesses: Point out 2-4 areas that need improvement
            • Industry fit: Based on the skills and experience, suggest 2-3 suitable industry sectors or job roles
            
            2. CONTENT IMPROVEMENTS
            • Achievements: Suggest how to better quantify results (provide 2-3 examples of how to reword vague statements)
            • Skills presentation: Advise on better organization or presentation of technical skills
            • Missing skills: Identify any critical skills that seem to be missing based on the experience described
            
            3. FORMAT SUGGESTIONS
            • Structure: Suggest optimal resume sections and ordering
            • Length: Advise on appropriate length based on experience level
            • Readability: Provide tips to improve scannability
            
            4. ATS OPTIMIZATION
            • Keywords: Suggest 5-7 additional keywords to include for better ATS matching
            • Formatting pitfalls: Identify any elements that could harm ATS parsing
            • File format recommendations
            
            Be extremely specific and actionable in your suggestions. Provide concrete examples where possible.
            Focus on transformative improvements rather than minor tweaks.
            """

### Tool-1: Agent for resume Analysis:

In [127]:
@tool
def analyze_resume(skills: str, education: str, experience: str):
    "This tool is use in case user is asking for resume analysis"
    prompt = resume_analysis_prompt.format({"skills": skills, "education": education, "experience": experience})
    try:
        analysis =  llm.invoke(prompt).content
    except:
        # Basic strengths analysis
        strengths = []
        if len(skills) >= 5:
            strengths.append("Good range of technical skills")
        if len(experience) >= 3:
            strengths.append("Solid work experience")
        if any("machine learning" in skill.lower() or "ai" in skill.lower() for skill in skills):
            strengths.append("Valuable AI/ML skills that are in high demand")
        
        analysis += "Strengths:\n"
        for strength in strengths or ["Resume contains some relevant skills"]:
            analysis += f"• {strength}\n"
        
        # Basic weaknesses analysis
        weaknesses = []
        if len(skills) < 5:
            weaknesses.append("Limited range of technical skills listed")
        if not any("python" in skill.lower() for skill in skills):
            weaknesses.append("Python (a widely used programming language) not explicitly listed")
        
        analysis += "\nWeaknesses:\n"
        for weakness in weaknesses or ["Consider adding more specific technical skills"]:
            analysis += f"• {weakness}\n"
        
        # Content improvements
        analysis += "\nCONTENT IMPROVEMENTS\n\n"
        analysis += "• Consider quantifying your achievements with specific metrics\n"
        analysis += "• Organize skills by category (programming languages, frameworks, tools)\n"
        analysis += "• Focus on highlighting relevant skills for your target roles\n"
        
        # Format suggestions
        analysis += "\nFORMAT SUGGESTIONS\n\n"
        analysis += "• Use a clean, ATS-friendly format with clear section headings\n"
        analysis += "• Ensure consistent formatting (bullet points, dates, etc.)\n"
        analysis += "• Keep resume to 1-2 pages maximum\n"
        
        # ATS optimization
        analysis += "\nATS OPTIMIZATION\n\n"
        analysis += "• Use keywords from job descriptions in your resume\n"
        analysis += "• Save your resume as a PDF to maintain formatting\n"
        analysis += "• Avoid tables, headers/footers, and images that can confuse ATS systems\n"
    
    return analysis

In [128]:
# agent_executor = create_react_agent([analyze_resume],llm,agent="zero-shot-react-description", verbose=True)
agent_executor = create_react_agent(llm, [analyze_resume], debug=True)

In [129]:
temp_path  = r"U:\generative_ai_projects\job_search_assistant\atharva_mishra_resume.pdf"

In [130]:
def extract_text(temp_path):
    extracted_text = ""
    if temp_path.endswith('.pdf'):
        with open(temp_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                extracted_text += page.extract_text() + "\n"
    return extracted_text

In [131]:
extracted_text = extract_text(temp_path)

In [132]:
resume_data = resume_parser.parse_resume(extracted_text)

In [133]:
# resume_data

In [ ]:
skills = ", ".join(resume_data.get("skills", []))

education_list = resume_data.get("education", [])
education = "\n".join([f"- {edu}" for edu in education_list])

experience_list = resume_data.get("experience", [])
experience = "\n".join([f"- {exp}" for exp in experience_list])

In [135]:
result = analyze_resume.invoke({
    "skills": skills,
    "education": education,
    "experience": experience
})
print(result)

## 1. OVERALL ASSESSMENT  

| Category | What the resume shows | What it should aim for |
|----------|-----------------------|------------------------|
| **Strengths** | 1. **Strong technical stack** – Python, PyTorch, TensorFlow, scikit‑learn, OpenCV, SQL, MongoDB, Flask, Git.<br>2. **Project‑centric experience** – Image‑captioning app, flight‑price predictor, EDA pipelines.<br>3. **Formal education & certifications** – B.E. from Oriental Institute + “Full‑Stack Data Science” (iNeuron).<br>4. **Bilingual** – English & Hindi (useful for global teams). | Keep the stack current (e.g., add cloud, Docker, CI/CD) and highlight business impact. |
| **Weaknesses** | 1. **Messy formatting** – duplicated skills, random line breaks, missing dates. <br>2. **Vague achievements** – “proficient in data preprocessing” without numbers or outcomes. <br>3. **Incomplete experience section** – no clear job titles, responsibilities, or dates. <br>4. **Missing soft/transferable skills** – leadership, commun

In [136]:
input_message = {
    "role": "user",
    "content": f"Please analyze this resume with the following details:\n\nSkills: {skills}\n\nEducation:\n{education}\n\nExperience:\n{experience}"
}

response = agent_executor.invoke({"messages": [input_message]})
print(response)

[values] {'messages': [HumanMessage(content='Please analyze this resume with the following details:\n\nSkills: Pytorch, data science, sql, mongodb, Hindi, Python, NLTK, scikit-learn, numpy, Commputer Vision, Matplotlib, keras, iNeuron, Genetic Variants, computer vision, Machine Learning, Spyder, pandas, flask, machine learning, html, deep learning, tensorflow, pytorch, css, git, VScode, Oriental Institute Of Science Of Technology, python, Keras, EDA\n\nEducation:\n- h, Hindi.\nCertifications: Full Stack Data Science (iNeuron), Python(Durgasoft)  Mar 2013 - Mar 2015\nOriental Institute Of Science Of Technology (Bachelor Of Engineering)                                                    Aug 2015 - Oct 2020Khu\n- Institute\n\nExperience:\n- 15\nOriental Institute Of Science Of Technology (Bachelor Of Engineering)                                                    Aug 2015 - Oct 2020Khusbu Yadav Inter Collage (12th)Developed an Image Captioning Web Application that generates descriptive ca

In [137]:
type(response)

dict